<a href="https://colab.research.google.com/github/andreacangiani/NSPDE-ANA22/blob/main/python/C6_after_the_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The FEniCS Project

The [FEniCS Project](https://colab.research.google.com/drive/1UX17QtYCpfLQhdu_Z8c3b3VlEh1Onjf8#scrollTo=u-kZtjlmAhjr&line=1&uniqifier=1) is an open-source software project aimed at creating an automated workflow for computational mathematical modelling based on the Finite Element Method (FEM).

The latest version of the FEniCS project, FEniCSx, consists of several building blocks: 
* DOLFINx is the FEM high performance C++ backend of FEniCSx, implementing structures such as meshes, function spaces and functions. DOLFINx also performs finite element assembly and mesh refinement algorithms. Finally, it interfaces to linear algebra solvers and data-structures, such as [PETSc](https://petsc.org/release/).
* UFL is a high-level form language for describing variational formulations with a high-level mathematical syntax
* FFCx is the form compiler of FEniCSx; given variational formulations written with UFL, it generates efficient C code.
* Basix is the finite element backend of FEniCSx, responsible for generating finite element basis functions.


As many other open-source software, FEniCS uses other packages while carrying out specific tasks of the FEM pipeline. A few notable dependencies of FEniCS are:

*  PETSc (and its Python wrapping petsc4py) for linear algebra solvers (and much more, such as nonlinear solvers and time stepping);
*    SLEPc (and its Python wrapping slepc4py) for solution of eigenvalue problems;
*    MPI for parallel computing;
*    ParMETIS and SCOTCH for mesh partitioning in parallel computing;
*    Gmsh for generation of complex meshes;
*    numpy for matrix/vector manipulation from Python;
*    plotly/pyvista for plotting meshes and solutions.


More details can be found in the original tutorial: 
* Hans Petter Langtangen, Anders Logg, *[Solving PDEs in Python: The FEniCS Tutorial I](https://link.springer.com/book/10.1007/978-3-319-52462-7)*, Simula SpringerBriefs on Computing, Springer, 2016whose code is found [here](https://jorgensd.github.io/dolfinx-tutorial/).

# FEniCS modules

In this tutorial we will explicitly use only a few libraries, namely numpy, petsc4py, UFL, dolfinx. However, all FEniCS software components (and many of the dependencies listed above) will be used under the hood 

We start by importing all modules which we require.

We do this through the python try/except blocks: 
* the try block lets you test a block of code for errors.
* the except block lets you handle the error.

In [ ]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/20faf6e/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx


In [ ]:
try:
    import pyvista
except ImportError:
    !pip3 install itkwidgets==0.32.1 pyvista==0.33.2
    import pyvista
finally:
    import google.colab
    google.colab.output.enable_custom_widget_manager()

In [ ]:
try:
    import multiphenicsx
except ImportError:
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@bdc5d58"
    import multiphenicsx

In [4]:
import dolfinx.fem
import dolfinx.mesh
import mpi4py
import multiphenicsx.io
import numpy as np
import petsc4py
import ufl

# Tutorial 1: solving a diffusion problem in 1D

We consider the model boundary value problem:
$$
\left\{
\begin{array}{l}
- u'' = 2, & x \in I= (0, 1),\\
u(0) = 0,\\
u(1) = 1.
\end{array}
\right.
$$

**Task 1: create a mesh.**

`dolfinx.mesh` provide some built-in functions to generate simple meshes, and in particular `create_unit_interval` for an equispaced mesh on the unit interval $I$. 

 Create the uniform mesh with 10 cells:

In [5]:
mesh = dolfinx.mesh.create_unit_interval(mpi4py.MPI.COMM_WORLD, 10)

Note that `dolfinx.mesh` requires that we supply the MPI-communicator. This is to specify how we would like the program to behave in parallel. With:
* MPI.COMM_WORLD we create a single mesh, whose data is distributed over the number of processors we would like to use. 
* MPI.COMM_SELF we create a separate mesh on each processor

We can obtain an interactive plot of the domain using `plotly`.

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

A mesh is made by a set of points and a set of subintervals that connect them:

In [ ]:
points = mesh.geometry.x
points

(Note that `dolfinx` developers decided to store points as vectors in $\mathbb{R}^3$, regardless of the actual ambient space dimension!)

In [ ]:
connectivity_cells_to_vertices = mesh.topology.connectivity(mesh.topology.dim, 0)
connectivity_cells_to_vertices

In [ ]:
num_cells = len(connectivity_cells_to_vertices)
num_cells

We can have a look at each cell  by using a `for` loop. Each cell is assigned an unique ID and (in 1D) it is uniquely defined by two vertices, which correspond to the endpoints of the subinterval.

In [ ]:
for c in range(num_cells):
    # Print the ID of the current cell
    print("Cell ID", c, "is defined by the following vertices:")
    # Print the vertices of the current cell
    for v in connectivity_cells_to_vertices.links(c):
        print("\t" + "Vertex ID", v, "is located at x =", points[v][0])

Next, we identify the IDs corresponding to boundary nodes. We use the

`dolfinx.mesh` function `locate_entities_boundary`. It requires the following inputs:
 * the first argument is the mesh,
 * the second argument represent the topological dimension of the mesh entities which we are interested in. In 1D, `mesh.topology.dim` is equal to 1, and entities of topological dimension 1 are the cells (subintervals), while `mesh.topology.dim - 1` is equal to 0, and entities of topological dimension 0 are the vertices of mesh. 
 * the third argument is a condition (i.e., a function that returns either `True` or `False`) on the coordinates `x`, which are stored as a vector. Since we are interested in finding the vertex located at $x = 0$, we may think of using `x[0] == 0` as a condition: however, due to floating point arithmetic, it is safer to use $\left|x - 0\right| < \varepsilon$, where $\varepsilon$ is a small number, which may be written as `np.isclose(x[0], 0.0)`.

In [ ]:
tdim = mesh.topology.dim
fdim = tdim - 1

In [ ]:
left_boundary_entities = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: np.isclose(x[0], 0.0))
left_boundary_entities

In [ ]:
right_boundary_entities = dolfinx.mesh.locate_entities(
    mesh, fdim, lambda x: np.isclose(x[0], 1.0))
right_boundary_entities

**Task 2: create FEM space.**

We define the finite element function space $V_h$ using $\mathbb{P}_2$ Lagrange elements.

This is obtained using the `FunctionSpace` class of `dolfinx.fem`.

The first argument specifies the mesh. The second the type of FE space. To define the standard (conforming) Lagrange elements we input `"CG"`. Using instead `"Lagrange"` or `"P"` yields the same space.

In [51]:
Vh = dolfinx.fem.FunctionSpace(mesh, ("CG", 2))
Vh

FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', interval, 1), dim=1), 0), FiniteElement('Lagrange', interval, 1))

Store the dimension of the space:

In [ ]:
Vh_dim = Vh.dofmap.index_map.size_local
Vh_dim

Once the FE space is at hand, we introduce ufl symbols to define the trial and test functions for our weak formulation:

In [53]:
uh = ufl.TrialFunction(Vh)
vh = ufl.TestFunction(Vh)

**Task 3:** Set up FEM system

Now we are ready to define the FEM using the ufl capability.
* `uh.dx(0)` corresponds to $\frac{\partial u}{\partial x}$, where the argument `0` to `dx` means to take the derivative with respect to the first space coordinate (the only one of interest in this case).
* `ufl.dx` provides a measure for integration over the domain. Integration will automatically occur over the entire domain.

In [54]:
a = uh.dx(0) * vh.dx(0) * ufl.dx

F = 2 * vh * ufl.dx

**Task 4:** Apply boundary conditions

It remains to implement the boundary conditions. To do so we:
* determine the degree of freedom that corresponds to the boundary vertices.
* define a `Constant` equal to `0` and a `Constant` equal to `1` corresponding to the values on the boundary.
* create a list containing the Dirichlet boundary conditions (two in this case), that is the constraints on the FE function DoF:

In [ ]:
left_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, fdim, left_boundary_entities)
left_boundary_dofs

In [ ]:
right_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, fdim, right_boundary_entities)
right_boundary_dofs

In [33]:
zero = dolfinx.fem.Constant(mesh, 0.)
one = dolfinx.fem.Constant(mesh, 1.)

In [ ]:
bcs = [dolfinx.fem.dirichletbc(zero, left_boundary_dofs, Vh), dolfinx.fem.dirichletbc(one, right_boundary_dofs, Vh)]
bcs

Alternatively use boundary data function automaticallly: 

* Define the boundary data through a function, for instance:

In [55]:
uD = dolfinx.fem.Function(Vh)
uD.interpolate(lambda x: x[0])

* Create facet to cell connectivity required to determine boundary facets

In [ ]:
mesh.topology.create_connectivity(fdim, tdim)

print(dolfinx.mesh.compute_boundary_facets(mesh.topology))

# np.flatnonzero gives indices of the elements that are non-zero
boundary_points = np.flatnonzero(dolfinx.mesh.compute_boundary_facets(mesh.topology))

boundary_points

* We can now apply the boundary conditions:

In [ ]:
boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, fdim, boundary_points)
bcs = dolfinx.fem.dirichletbc(uD, boundary_dofs)
bcs

**Task 5:** Solve the FEM system

In order to solve the FEM system, we go through the following steps:

* `dolfinx.fem` provides a `Function` class to store the solution of a finite element problem:
* solve the discrete problem allocating a new `LinearProblem` (which uses `PETSc`), providing as input the bilinear form `a`, the linear functional `F`, the boundary conditions `bcs`, and where to store the solution. Further solver options can also be passed to `PETSc`.

In [58]:
solution = dolfinx.fem.Function(Vh)

In [ ]:
problem = dolfinx.fem.petsc.LinearProblem(
    a, F, bcs=[bcs], u=solution,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()

print(solution.vector.array)

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "u_h")

# Compute error

**Task 6:** compute the $L^2$ and $H^1$ errors.

The exact solution is:
$$ u(x) = - x^2 + 2 x.$$

The $L^2(I)$ norm of the error $u - u_h$ is defined as:
$$ e_h^2 = \int_I \left(u(x) - u_h(x)\right)^2 \ \mathrm{d}x.$$

In order to evaluate the error, we first need to define a symbolic representation in `ufl` of the exact solution $u(x)$. To this end, we need to define a symbol for the coordinate `x` ...

In [87]:
xyz = ufl.SpatialCoordinate(mesh)
x = xyz[0]

and then we can define a symbolic expression in `ufl` for the exact solution $u$:

In [88]:
exact_solution = - x**2 + 2 * x

Hence we can define a symbolic expression in `ufl` for the integral defining the error:

In [89]:
error_L2squared_ufl = (exact_solution - solution)**2 * ufl.dx

Finally, we evaluate the error using the `dolfinx.fem` function `assemble_scalar`:

In [ ]:
error_L2squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(error_L2squared_ufl))
error_L2squared

Note that, given that we are using quadratic elements, we expect the error to be zero!

Similarly we can compute the H1 seminorm error:

In [ ]:
error_H1squared_ufl = (exact_solution.dx(0) - solution.dx(0))**2 * ufl.dx
error_H1squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(error_H1squared_ufl))
error_H1squared